In [1]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import binarize
import sklearn
from sklearn.svm import LinearSVC
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import random

In [2]:
location='/Users/ashishtrehan/Downloads/rstudio-export/'
df=pd.read_csv(location+str('FLU_Challenge_vietnam.csv'))

In [22]:
x = kmer_count(df['seqs'][0],100,10)

In [3]:
def length(row):
    x = len(row['seqs'])
    return x

df['length']=df.apply (lambda row: length (row),axis=1)

In [34]:
df['length'].min()

271

In [100]:
def sequence(row,n):
    s=row['seqs']
    k=len(s)-2*n
    klist = []
    for i in range(k):
        kmer=s[i:i+n]
        if not(kmer in klist) and (kmer in s[i+n:]):
            klist.append(kmer)
    return klist
        

In [101]:
df['sequence']=df.apply (lambda row: sequence(row,3),axis=1)

In [91]:
for index,x in df.iterrows():
    for z in x['sequence']:
        #print z

IndentationError: expected an indented block (<ipython-input-91-cf325960ccbc>, line 3)

In [106]:
seq=[]
for index,x in df.iterrows():
    for z in x['sequence']:
        seq.append(str(z))
    

In [112]:
def olpcount(string,pattern,casesensitive=True):
    if casesensitive != True:
        string  = string.lower()
        pattern = string.lower()
    l = len(pattern)
    ct = 0
    for c in range(0,len(string)):
        if string[c:c+l] == pattern:
            ct += 1
    return ct

In [124]:
for index,x in df.iterrows():
    for z in x['sequence']:
        c = olpcount(df['seqs'][0],z)
        

In [129]:
len(df['sequence'][0])
    #print len(x),x
    

64

In [134]:
from Bio import SeqIO

In [136]:
#for index, letter in enumerate(df['seqs'][0]):
    #print("%i %s" % (index, letter))

In [4]:
gencode = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_', 'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W'}
 
# a function to translate a single codon
def translate_codon(codon):
    return gencode.get(codon.upper(), 'x')
 
# a function to split a sequence into codons
def split_into_codons(row, frame):
    dna=row['seqs']
    codons = []
    for i in range(frame - 1, len(dna)-2, 3):
        codon = dna[i:i+3]
        codons.append(codon)
    return ' '.join(codons)
 
# a function to translate a dna sequence in a single frame
def translate_dna_single(dna, frame=1):
    codons = split_into_codons(dna, frame)
    amino_acids = ''
    for codon in codons:
        amino_acids = amino_acids + translate_codon(codon)
    return amino_acids
 
# a function to translate a dna sequence in 3 forward frames
def translate_dna(dna):
    all_translations = []
    for frame in range(1,4):
        all_translations.append(translate_dna_single(dna, frame))
    return all_translations

In [5]:
df['codons']=df.apply (lambda row: split_into_codons(row,1),axis=1)

In [6]:
df.head()

,ID,seqs,length,codons
0,0,agcaaaagcaggggtccaatctgtcaaaatggaaaaaatagtgctt...,1779,agc aaa agc agg ggt cca atc tgt caa aat gga aa...
1,0,atggagaaaatagtgcttcttcttgcaatagttagccttgttaaaa...,1704,atg gag aaa ata gtg ctt ctt ctt gca ata gtt ag...
2,0,atggagaaaatagtgcttcttcttgcaatggtcagccttgttaaaa...,1704,atg gag aaa ata gtg ctt ctt ctt gca atg gtc ag...
3,0,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,1704,atg gag aaa ata gtg ctt ctt ctt gca ata gtc ag...
4,0,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,1704,atg gag aaa ata gtg ctt ctt ctt gca ata gtc ag...


In [7]:
class TreeFS(sklearn.base.BaseEstimator):
    
    def __init__(self, n_features=100):
        self.forest = ExtraTreesClassifier(n_estimators=100)
        self.n_features = n_features
    
    def fit(self, df, y=None):
        self.forest.fit(df, y)
        self.importances = self.forest.feature_importances_
        self.indices = np.argsort(-self.importances)[0:self.n_features]
        self.indices.sort()
        return self
        
    def transform(self, X, y=None):   
        return X[:, self.indices]
        
class Ngrams(sklearn.base.BaseEstimator):
    def __init__(self, ngrams=(1,1),min_df=1, name="ngram"):
        self.min_df = min_df
        self.ngrams = ngrams
        self.name = name
        
    def get_feature_names(self):
        return self.feature_names_
    
    def _set_feature_names(self, features):
        return [(self.name, x) for x in features]
        
        
    def fit(self, df,y=None):
        self.tfidf = CountVectorizer(ngram_range=self.ngrams, min_df=self.min_df, max_df=0.9,token_pattern="[a-zA-Z]{2,15}")
                                    #vocabulary=['New Girl'])
                                     #stop_words="english")
                                     #token_pattern="[a-zA-Z]{2,15}")
        self.tfidf.fit(df)
        self.feature_names_ = self._set_feature_names(self.tfidf.get_feature_names())
        return self
    
    def transform(self, df,y=None):
        return binarize(self.tfidf.transform(df).todense())

class OneGrams(Ngrams):
    
    def __init__(self, min_df=1):
        super(OneGrams, self).__init__(min_df=min_df, name="onegram")
        

class MGrams(Ngrams):
    
    def __init__(self, ngrams=(2,2), min_df=1):
        super(MGrams, self).__init__(ngrams=ngrams,min_df=min_df,name="mgram")
        
    def _set_feature_names(self, features):
        return [(self.name, "[, .:;/-]+".join(x.split(" "))) for x in features]
    

class FeaturePipeline(FeatureUnion):
    
    def __init__(self, transformer_list, n_jobs=1, transformer_weights=None):
        super(FeaturePipeline, self).__init__(transformer_list, n_jobs, transformer_weights)
        
    def get_features(self):
        feature_names = []
        for name, trans in self.transformer_list:
            feature_names.extend(trans.get_feature_names())
        return feature_names

forest = TreeFS(n_features=800)    
svm = LinearSVC(fit_intercept=False, random_state=1)
fp = FeaturePipeline(sklearn.pipeline._name_estimators([OneGrams(min_df=2), MGrams((2,2), min_df=2), MGrams((3,3),min_df=2)]))
p = Pipeline([("features",fp),("forest",forest),("svm",svm)])
print p

Pipeline(steps=[('features', FeaturePipeline(n_jobs=1,
        transformer_list=[('onegrams', OneGrams(min_df=2)), ('mgrams-1', MGrams(min_df=2, ngrams=(2, 2))), ('mgrams-2', MGrams(min_df=2, ngrams=(3, 3)))],
        transformer_weights=None)), ('forest', TreeFS(n_features=800)), ('svm', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=False,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=1, tol=0.0001,
     verbose=0))])


In [8]:
df=df.fillna(3)
#df_train[df['ID']]

In [9]:
df.groupby(['ID']).size()

ID
0    467
1     48
3    115
dtype: int64

In [11]:
msk = np.random.rand(len(df)) < 0.8
df_training=df[(df['ID']<3)]
msk = np.random.rand(len(df_training)) < 0.8
df_train=df_training[msk]
df_test=df_training[~msk]
print "training set has {0} observations and test set has {1}".format(len(df_train),len(df_test))

training set has 421 obversations and test set has 94


In [12]:
p.fit(df_train["codons"], 
      df_train["ID"])

Pipeline(steps=[('features', FeaturePipeline(n_jobs=1,
        transformer_list=[('onegrams', OneGrams(min_df=2)), ('mgrams-1', MGrams(min_df=2, ngrams=(2, 2))), ('mgrams-2', MGrams(min_df=2, ngrams=(3, 3)))],
        transformer_weights=None)), ('forest', TreeFS(n_features=800)), ('svm', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=False,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=1, tol=0.0001,
     verbose=0))])

In [13]:
x=list(reversed(sorted(zip(p.named_steps["svm"].coef_.reshape((-1,)), 
           np.array([feature[1] for feature in p.named_steps["features"].get_features()])[p.named_steps["forest"].indices]))))

In [14]:
print "Actual,Predicted,First 10 of Sequence"
for x,z,c in zip(df_test['ID'],p.predict(df_test['codons']),df_test['seqs']):
    print x,z,c[:10]

Actual,Predicted,First 10 of Sequence
0.0 0.0 agcaaaagca
0.0 0.0 atggagggaa
0.0 0.0 atggagagaa
0.0 0.0 ccaatctgtc
0.0 0.0 atggagaaaa
0.0 0.0 agcaaaagca
0.0 0.0 atggagaaaa
0.0 0.0 agcaaaagca
0.0 0.0 agcaaaagca
0.0 0.0 gatcagattt
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 agcaaaagca
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 agcaaaagca
0.0 0.0 aatctgtcaa
0.0 0.0 agcaaaagca
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 aaaatggaga
0.0 0.0 aaaatggata
0.0 0.0 agcaaaagca
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 ttcaatttgt
0.0 0.0 aaatagtgct
0.0 0.0 agcaaaagca
0.0 1.0 tcaaaatgga
0.0 0.0 ggttcaatct
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 agcaaaagca
0.0 0.0 agcaaaagca
0.0 0.0 agcgaaagca
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 agcaaaagca
0.0 0.0 ggttcaatct
0.0 0.0 ggttcaatct
0.0 0.0 atggagaaaa
0.0 1.0 ggttcaatct
0.0 0.0 atggagaaaa
0.0 0.0 atggagaaaa
0.0 0.0 agcaaaagcc
0.0 0.0 aaaatggaga
0.0 0.0 atggagaaaa
0.0 0.0 atgg

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
accuracy_score(df_test['ID'],p.predict(df_test['codons']))

0.92553191489361697

In [34]:
from nltk.text import Text  
textList = Text(df[(df['ID']<3)]['codons'])

In [35]:
textList.concordance('att')

No matches


In [38]:
df['codons'].to_csv('/Users/ashishtrehan/Downloads/text.txt')

In [26]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [33]:
from gensim.models import word2vec
print "Training model..."
#model = word2vec.Word2Vec(textList, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

IndentationError: unexpected indent (<ipython-input-33-01b0d755e78e>, line 4)

In [45]:
from gensim.models import word2vec

ImportError: cannot import name utils